In [5]:
# pip install -r dependencies.txt

In [6]:
import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment
import time

In [7]:
# Carregar dados CSV
diplomatas = pd.read_csv('../resources/aed_alocacao_recursos - diplomatas.csv')
cidades = pd.read_csv('../resources/aed_alocacao_recursos - cidades.csv')

# print(diplomatas.info())
# print(cidades.info())

In [8]:
def get_posto_trabalho(id, cidades):
  return cidades.loc[cidades['posto'] == id].to_dict(orient='records')[0]

get_posto_trabalho('P1', cidades)

{'cidade': 'C1', 'posto': 'P1', 'classificacao': 'A'}

In [9]:
def calcular_custo_aresta(diplomata, posto, cidades):
    # print(f"Calculando custo da aresta {diplomata.id} -> {posto.posto}")
    # Se tiver 12 anos fora, deve retornar
    if diplomata.tempo_fora >= 144:
        if posto.classificacao == '*':
            return 0.0
        else:
            return 999999999.99
    # Se tiver 2 anos na lotação atual, deve trocar de lotacao
    if diplomata.tempo_na_lotacao >= 24 and diplomata.lotacao == posto.posto:
        return 999999999.99

    # verificar se a origem á compatível com o destino. Ex. a só pode ir para B, C ou D
    if diplomata.lotacao != 'P0':
        posto_atual = get_posto_trabalho(diplomata.lotacao, cidades)
        # print('Posto atual {}'.format(posto_atual))
        if diplomata.lotacao != posto.posto and posto.classificacao == 'A':
            if posto_atual.get('classificacao')  == 'A':
                return 999999999.99
    elif diplomata.pedagio < 24 and posto.classificacao == 'A':
        # Se estiver a menos de 2 anos no Brasil, não pode ir para um posto 'A'
        return 999999999.99

    if posto.classificacao == 'A':
        return 0.0

    if posto.classificacao == 'B':
        return 10.0

    if posto.classificacao == 'C':
        return 100.0

    if posto.classificacao == 'D':
        return 1000.0
    
    return 999999999.99

In [10]:
def alocar_diplomatas(matriz_custos):
  """
  Resolve o problema de alocação usando o algoritmo húngaro.

  :param matriz_custos: Matriz de custos (2D list ou numpy array)
  :return: Tupla (lista de alocações, custo total)
  """
  # Usando a função `linear_sum_assignment` da biblioteca scipy
  matriz_custos = np.array(matriz_custos)
  linhas, colunas = linear_sum_assignment(matriz_custos)
  custo_total = matriz_custos[linhas, colunas].sum()

  return linhas, colunas, custo_total

In [11]:
def calcular_custos_arestas(diplomatas, cidades):
    diplomatas_tratado = diplomatas.sort_values("tempo_servico", ascending = False)
    pesos_arestas = []
    for d, row in diplomatas_tratado.iterrows():
        pesos_diplomata = []
        for i, posto in cidades.iterrows():
            pesos_diplomata.append(calcular_custo_aresta(row,posto,cidades))
        pesos_arestas.append(pesos_diplomata)

    return pesos_arestas

In [12]:
# Exemplo de uso
pesos_arestas = calcular_custos_arestas(diplomatas,cidades)
linhas, colunas, custo = alocar_diplomatas(pesos_arestas)
diplomatas_tratado = diplomatas.sort_values("tempo_servico", ascending = False)
print("Alocações (diplomata -> posto):")
for i, j in zip(linhas, colunas):
  print(f"Diplomata {diplomatas_tratado.iloc[i].id} -> Posto {cidades.iloc[j].posto}")

print("Custo total:", custo)

Alocações (diplomata -> posto):
Diplomata D30 -> Posto P0
Diplomata D29 -> Posto P1
Diplomata D28 -> Posto P2
Diplomata D27 -> Posto P3
Diplomata D26 -> Posto P4
Diplomata D25 -> Posto P5
Diplomata D24 -> Posto P6
Diplomata D23 -> Posto P7
Diplomata D22 -> Posto P8
Diplomata D21 -> Posto P9
Diplomata D20 -> Posto P10
Diplomata D19 -> Posto P11
Diplomata D18 -> Posto P12
Diplomata D17 -> Posto P13
Diplomata D16 -> Posto P14
Diplomata D15 -> Posto P16
Diplomata D14 -> Posto P15
Diplomata D13 -> Posto P17
Diplomata D12 -> Posto P18
Diplomata D11 -> Posto P19
Diplomata D10 -> Posto P20
Diplomata D9 -> Posto P21
Diplomata D8 -> Posto P22
Diplomata D7 -> Posto P23
Diplomata D6 -> Posto P24
Diplomata D5 -> Posto P25
Diplomata D4 -> Posto P26
Diplomata D3 -> Posto P27
Diplomata D2 -> Posto P28
Diplomata D1 -> Posto P29
Custo total: 1000.0


In [27]:
def gerar_base_diplomatas(qtd_diplomatas, qtd_postos):
  # Geração da lotação
  ids = []
  pedagios = []
  lotacoes = []
  tempos_servico = []
  tempos_lotacao = []
  tempos_fora = []

  for p in range(qtd_diplomatas):
    while True:
      lotacao = 'P' + str(np.random.randint(qtd_postos))
      if lotacao == 'P0' or lotacao not in lotacoes:
        break

    tempo_servico = np.random.randint(300)  # tempo de serviço em meses
    pedagio = 0                             # tempo no Brasil em anos
    tempo_na_lotacao = 0                    # tempo na lotação em meses
    tempo_fora = 0                          # tempo ininterrupro fora do Brasil em meses
    if lotacao == 'P0': # Ele está no Brasil
      pedagio = np.random.randint(30)
      tempo_na_lotacao = 0
      tempo_fora = 0
    else:
      pedagio = 0
      tempo_na_lotacao = np.random.randint(25)
      tempo_fora = tempo_na_lotacao + np.random.randint(97,122)

    ids.append('D' + str(p+1))
    pedagios.append(pedagio)
    lotacoes.append(lotacao)
    tempos_servico.append(tempo_servico)
    tempos_lotacao.append(tempo_na_lotacao)
    tempos_fora.append(tempo_fora)

  d = {
    'id': ids,
    'pedagio': pedagios,
    'lotacao': lotacoes,
    'tempo_servico': tempos_servico,
    'tempo_na_lotacao': tempos_lotacao,
    'tempo_fora': tempos_fora
  }
  return pd.DataFrame(data=d, index=[*range(qtd_diplomatas)])

In [14]:
# dpls = gerar_base_diplomatas(150,175)

# dpls.loc[(dpls['lotacao'] != 'P0') & (dpls['pedagio'] != 0)]

In [15]:
def gerar_base_postos(qtd_postos):
  postos = ['P0']
  classificacoes = ['*']
  tipos_classificacoes = ['A','B','C','D']

  for p in range(qtd_postos):
    posto = 'P' + str(p+1)
    postos.append(posto)
    classificacoes.append(tipos_classificacoes[np.random.randint(4)])

  d = {
    'posto': postos,
    'classificacao': classificacoes
  }

  return pd.DataFrame(data=d, index=[*range(qtd_postos+1)])

In [28]:
def rodar_teste(qtd_diplomatas, multiplicador_postos = 1.5):
  # Gerar base de testes
  qtd_postos = multiplicador_postos * qtd_diplomatas
  t_postos = gerar_base_postos(int(qtd_postos))
  t_diplomatas = gerar_base_diplomatas(qtd_diplomatas, qtd_postos)

  # Executar alocações
  inicio = time.time()
  t_pesos_arestas = calcular_custos_arestas(t_diplomatas,t_postos)
  linhas, colunas, custo = alocar_diplomatas(t_pesos_arestas)
  fim = time.time()
  #preparar dados de retorno
  # pedagio,lotacao,tempo_servico,tempo_na_lotacao,tempo_fora
  t_diplomatas_tratado = t_diplomatas.sort_values("tempo_servico", ascending = False)
  t_diplomatas_tratado = t_diplomatas_tratado.merge(t_postos, how='inner', left_on='lotacao',right_on='posto')
  t_diplomatas_alocados = []
  t_postos_alocados = []
  t_classificacoes_atuais = []
  t_classificacoes_alocadas = []
  t_tempos_servicos = []
  t_pedagios = []
  t_lotacoes_anteriores = []
  t_tempos_nas_lotacoes = []
  t_tempos_fora = []

  for i, j in zip(linhas, colunas):
    diplomata = t_diplomatas_tratado.iloc[i].id
    tempo_servico = t_diplomatas_tratado.iloc[i].tempo_servico
    tempo_lotacao = t_diplomatas_tratado.iloc[i].tempo_na_lotacao
    tempo_fora = t_diplomatas_tratado.iloc[i].tempo_fora
    pedagio = t_diplomatas_tratado.iloc[i].pedagio
    lotacao = t_diplomatas_tratado.iloc[i].lotacao
    alocacao = t_postos.iloc[j].posto
    classificacao_atual = t_diplomatas_tratado.iloc[i].classificacao
    classificacao_alocada = t_postos.iloc[j].classificacao
    '''
    print(f"""
      'diplomata': {diplomata},
      'tempo_servico': {tempo_servico},
      'tempo_na_lotacao': {tempo_lotacao},
      'tempo_fora': {tempo_fora},
      'tempo_sede': {pedagio},
      'lotacao_anterior': {lotacao},
      'classificacao_anterior': {classificacao_atual},
      'lotacao_alocada': {alocacao},
      'classificacao_alocada': {classificacao_alocada}
          """)
    '''
    t_diplomatas_alocados.append(diplomata)
    t_tempos_servicos.append(tempo_servico)
    t_tempos_nas_lotacoes.append(tempo_lotacao)
    t_tempos_fora.append(tempo_fora)
    t_pedagios.append(pedagio)
    t_lotacoes_anteriores.append(lotacao)
    t_postos_alocados.append(alocacao)
    t_classificacoes_atuais.append(classificacao_atual)
    t_classificacoes_alocadas.append(classificacao_alocada)

  d= {
    'diplomata': t_diplomatas_alocados,
    'tempo_servico': t_tempos_servicos,
    'tempo_na_lotacao': t_tempos_nas_lotacoes,
    'tempo_fora': t_tempos_fora,
    'tempo_sede': t_pedagios,
    'lotacao_anterior': t_lotacoes_anteriores,
    'classificacao_anterior': t_classificacoes_atuais,
    'lotacao_alocada': t_postos_alocados,
    'classificacao_alocada': t_classificacoes_alocadas
  }
  df = pd.DataFrame(data=d, index=[*range(qtd_diplomatas)])
  tempo = fim - inicio
  # print("Custo total:", custo)
  # print(f"Tempo de execução igual a {fim - inicio}")
  return df, custo, tempo

In [30]:
df, custo, tempo =  rodar_teste(150)

# dplm = gerar_base_diplomatas(100,150)
# pst = gerar_base_postos(150)

# all = dplm.merge(pst, how='inner', left_on='lotacao',right_on='posto')



In [ ]:
# média de tempo de serviço
# float(dplm['tempo_servico'].mean())/12

# Mediana do tempo de serviço
# float(dplm['tempo_servico'].median())/12

# dplm[['tempo_servico', 'tempo_na_lotacao', 'tempo_fora', 'pedagio']].describe()
# dplm[["posto","tempo_servico"]].groupby()

#####################
#### Calular a média de tempo de serviço por classe de posto antes e depois
####
####################
print(df[['classificacao_anterior', 'tempo_servico','tempo_na_lotacao','tempo_fora','tempo_sede']].groupby('classificacao_anterior').mean())
print(df[['classificacao_alocada', 'tempo_servico','tempo_na_lotacao','tempo_fora','tempo_sede']].groupby('classificacao_alocada').mean())

# df.loc[df['tempo_fora']>=144]

In [ ]:
# Rodar testes em massa

resultados = pd.DataFrame()
for q in range(50, 550, 100):
  df, custo, tempo =  rodar_teste(q)
  df.to_csv('../output/resultados_n_'+str(q)+'.csv', index=False)

KeyboardInterrupt: 